In [82]:
import anthropic

In [83]:
from dotenv import load_dotenv

In [84]:
load_dotenv()

True

In [85]:
client_anthropic = anthropic.Anthropic(api_key = os.getenv('CLAUDE_API_KEY'))

In [86]:
import os

In [87]:
import json

In [88]:
from supabase import create_client, Client

In [89]:
supabase_api_key = os.environ.get("SUPBASE_KEY")
supabase_url = os.environ.get("SUPABASE_URL")

In [90]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [124]:
def find_articles(date):
    result = supabase.table('research_assistant')\
        .select('id, event_id, news_date, output','topic')\
    .eq('news_date', date)\
        .order('created_at', desc=False)\
        .execute()

    return result.data

In [128]:
final_articles = find_articles('2026-02-21')

In [131]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux, a platform that explains AI and tech in under 60 seconds for working professionals and enthusiasts.
            
              TASK:
              Write one concise, high-signal news brief using ONLY the provided research notes.
            
              GLOBAL RULES:
              - Do not add facts that are not in the research notes.
              - Do not include URLs in the article body.
              - Keep language simple, clear, and concrete.
              - Avoid hype words and avoid clickbait that distorts facts.
              - If the input includes uncertainty/rumor, state that clearly in the first sentence.
              - No em dash characters.
            
              LENGTH RULES:
              - Headline: max 10 words.
              - Article body: 85 to 100 words (inclusive).
            
              STRUCTURE:
              1) Headline:
               -  Objective: Hook the reader instantly
               - Punchy, attention-grabbing and clickbait
               - Focus on the most surprising/important angle
            
              2) Body (single paragraph, 85-100 words):
              - Opening (core update + context)
              - Middle (most important factual details)
              - Closing (practical “why this matters now” implication grounded in provided facts)
            
              TOPIC-SPECIFIC STYLE (apply based on topic):
              - Funding:
                Focus on round/deal specifics and strategic signal; avoid generic market commentary.
              - Model announcements/enhancements:
                Focus on what changed vs before and who can use it now.
              - Workflow improvement:
                Focus on what changes in day-to-day work this week ("what changes Monday").
              - Report:
                Focus on the most important findings and one practical implementation takeaway.
              - Others:
                Focus on concrete significance and immediate relevance, avoid speculation.
            
              OUTPUT FORMAT (STRICT JSON ONLY):
              {{"headline":"...","output":"...","sources":[{{"name":"...","url":"..."}}]}}
            
              Research notes:
              {i['output']}
            
              Topic:
              {i['topic']}
              
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']
        article_json['topic'] = i['topic']

        print(article_json)

        final_summary_7.append(article_json)

        # save_article(article_json)

        print("✔️ Saved!")
        
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [135]:
claude_article_builder(final_articles[0:])

{'headline': "Fei-Fei Li's World Labs Raises $1 Billion Round", 'output': "World Labs closed a $1 billion equity financing round on February 18 to accelerate its spatial intelligence platform, Marble, which generates persistent, editable 3D worlds from text, images, or video. Autodesk led with a $200 million investment, joined by AMD Ventures, Nvidia, Sea Ltd., Emerson Collective, and Fidelity. The structure uses convertible preferred stock in a single close. Autodesk's stake signals a concrete integration push into professional 3D design workflows, targeting gaming, robotics, and visualization tools. Talks prior to closing pegged valuation around $5 billion, though no official post-money figure was disclosed.", 'sources': [{'name': 'Reuters', 'url': 'https://www.reuters.com/technology/ai-pioneer-feifei-lis-world-labs-raises-1-billion-funding-2026-02-18/'}, {'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/18/world-labs-lands-200m-from-autodesk-to-bring-world-models-into-3d

In [100]:
def save_article(article_json):
    supabase.table('hundred_word_articles').insert({
        'event_id': article_json['event_id'],
        'model_provider': article_json['model_provider'],
        'news_date': article_json['news_date'],
        'sources': json.dumps(article_json['sources']),
        'output': article_json['output'],
        'headline': article_json['headline']
    }).execute()

In [136]:
final_summary_7

[{'headline': 'Nvidia Ditches $100B Chip Deal for $30B OpenAI Stake',
  'output': "Nvidia is swapping its massive $100 billion multi-year chip supply agreement for a cleaner $30 billion direct equity investment in OpenAI. This shift makes Nvidia an actual owner, not just a supplier, in the AI giant currently valued at $830 billion. OpenAI's hunting for over $100 billion total this round, with SoftBank and Amazon joining the party. Plot twist: OpenAI plans to spend much of that fresh cash buying Nvidia chips anyway. This move signals Nvidia wants skin in the game beyond hardware sales, positioning itself as a stakeholder in AI's future rather than just the picks-and-shovels provider during the gold rush.",
  'sources': [{'name': 'The Star',
    'url': 'https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports'}],
  'model_provider': 'claude',
  'news_date': '2026-02-20',
  'event_id': '1274_2026-02-20

In [138]:
final_articles = [{'headline': 'Code Metal Hits Unicorn Status With $125M Series B',
  'output': "Code Metal closed a $125 million Series B led by Salesforce Ventures, reaching a $1.25 billion valuation. The round included Accel, B Capital, Shield Capital, and defense investor RTX Ventures. The company modernizes legacy defense software using AI-driven code translation tools with built-in verification for mission-critical systems. Funds will expand engineering and go-to-market partnerships with customers including the U.S. Air Force, RTX, and L3Harris. Former Tableau executive Ryan Aytay joins as President and COO. RTX Ventures' participation signals strategic alignment with defense primes seeking verifiable AI tooling for regulated, high-stakes environments.",
  'sources': [{'name': 'MLQ.ai',
    'url': 'https://mlq.ai/news/code-metal-lands-125m-series-b-to-modernize-legacy-defense-software-with-ai/'},
   {'name': 'Salesforce Ventures',
    'url': 'https://salesforceventures.com/perspectives/verified-code-for-mission-critical-systems/'},
   {'name': 'eWeek',
    'url': 'https://www.eweek.com/news/code-metal-valuation-ai-code-translation/'},
   {'name': 'WIRED',
    'url': 'https://www.wired.com/story/vibe-coding-startup-code-metal-raises-series-b-fundraising/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1319_2026-02-21',
  'topic': 'Funding'},
 {'headline': 'Peak XV Raises $1.3B in First Funds Post-Sequoia Split',
  'output': 'Peak XV Partners has closed $1.3 billion across three equity-focused venture funds targeting India and Asia, marking its first independent raise since separating from Sequoia. The capital spans India Seed, India Venture, and APAC funds, with global LPs including endowments and foundations participating. The firm will deploy the majority into India over the next two to three years, focusing on AI, fintech, consumer tech, and deeptech from seed to later stages. Peak XV now manages over $10 billion across 450+ portfolio companies and has backed more than 80 AI startups, signaling sustained conviction in the region despite recent senior-partner departures.',
  'sources': [{'name': 'TechCrunch',
    'url': 'https://techcrunch.com/2026/02/20/peak-xv-raises-1-3b-doubles-down-on-ai-as-global-vc-rivalry-in-india-heats-up/'},
   {'name': 'Moneycontrol',
    'url': 'https://www.moneycontrol.com/news/business/startup/peak-xv-partners-closes-1-3-billion-across-three-funds-targeting-india-apac-startups-13837875.html'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1301_2026-02-21',
  'topic': 'Funding'},
 {'headline': 'Gates, Wellcome Launch $60M AI Health Trial Fund',
  'output': "Gates Foundation, Novo Nordisk Foundation, and Wellcome today announced a $60 million grant program to evaluate mature AI health tools in low- and middle-income countries across sub-Saharan Africa, South Asia, and Southeast Asia. The Evidence for AI in Health initiative, delivered through J-PAL and APHRC, will fund randomized trials and implementation studies to assess efficacy, economic value, and acceptability of AI decision-support tools in primary care. Applications are now open. The program aims to generate real-world evidence guiding policy and deployment decisions in resource-constrained health systems, marking the second investment from the foundations' $300 million global health partnership.",
  'sources': [{'name': 'Gates Foundation',
    'url': 'https://www.gatesfoundation.org/ideas/media-center/press-releases/2026/02/ai-impact-health'},
   {'name': 'Abdul Latif Jameel Poverty Action Lab',
    'url': 'https://www.povertyactionlab.org/initiative/evidence-ai-health-evah-rfp'},
   {'name': 'APHRC',
    'url': 'https://aphrc.org/blogarticle/press-release-aphrc-joins-global-partners-to-deliver-the-60m-evidence-for-ai-in-health-evah-initiative-to-guide-responsible-use-of-ai-in-health-care/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1305_2026-02-21',
  'topic': 'Funding'},
 {'headline': "Anthropic's Claude Now Hunts Critical Bugs Across Codebases",
  'output': 'Anthropic launched Claude Code Security on February 20, 2026, a tool powered by Claude Opus 4.6 that autonomously scans entire codebases to find and prioritize high-severity vulnerabilities humans often miss. Unlike traditional pattern-based scanners, it performs end-to-end analysis, traces data flows across components, scores severity automatically, and suggests fixes without applying them. The research preview is available now for Enterprise and Team customers, with expedited access for open-source maintainers. Security teams gain faster triage and developers get guided remediation, addressing a critical gap in enterprise and open-source security workflows as vulnerabilities grow more complex.',
  'sources': [{'name': 'Fortune',
    'url': 'https://fortune.com/2026/02/20/exclusive-anthropic-rolls-out-ai-tool-that-can-hunt-software-bugs-on-its-own-including-the-most-dangerous-ones-humans-miss/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1300_2026-02-21',
  'topic': 'Model announcements/enhancements'},
 {'headline': 'Google Gemini Now Creates 30-Second AI Songs With Lyrics',
  'output': 'Google launched Lyria 3 in the Gemini app, letting users generate 30-second music tracks with lyrics and custom cover art from text or image prompts. Available in beta worldwide for users 18+, it adds auto-generated lyrics, enhanced control over style, vocals, and tempo, plus more realistic output compared to prior versions. Desktop access is live now, mobile coming soon. Creators can prototype video soundtracks faster or pair photos with prompts for shareable tracks. All outputs include SynthID watermarks for AI-origin verification. Google AI Plus, Pro, and Ultra subscribers get higher generation limits.',
  'sources': [{'name': 'Google',
    'url': 'https://blog.google/innovation-and-ai/products/gemini-app/lyria-3/'},
   {'name': 'TechCrunch',
    'url': 'https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1314_2026-02-21',
  'topic': 'Model announcements/enhancements'},
 {'headline': "India's Sarvam AI Unveils Models for 22 Languages",
  'output': "Sarvam AI launched two large language models, Sarvam-30B and Sarvam-105B, trained for all 22 Indian languages using a mixture-of-experts design with up to 128,000-token context windows. The company also debuted Vikram, an offline multilingual assistant that runs on feature phones, cars, and wearables for real-time voice conversations in local languages. Demos showed Hindi and Punjabi interactions for government-scheme guidance. Built with Nvidia and Yotta infrastructure under India's sovereign IndiaAI Mission, the models aim to challenge frontier AI with cost-efficient, edge-optimized reasoning and mark a major step toward homegrown, multilingual AI infrastructure.",
  'sources': [{'name': 'TechCrunch',
    'url': 'https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/'},
   {'name': 'Business Standard',
    'url': 'https://www.business-standard.com/technology/tech-news/ai-startup-sarvam-launches-two-made-in-india-large-language-models-126021801397_1.html'},
   {'name': 'ISTI Portal',
    'url': 'https://www.indiascienceandtechnology.gov.in/stihighlights/sarvam-ai-unveils-indigenous-multilingual-foundation-models-india-ai-impact-summit'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1316_2026-02-21',
  'topic': 'Model announcements/enhancements'},
 {'headline': 'Figma Make Pulls Live Research Data Into Prototypes',
  'output': 'Figma Make now supports external connectors via the Model Context Protocol, letting designers pull customer quotes, research findings, and analytics directly into prototype workflows using @mentions. Design teams can connect tools like Dovetail, Marvin, and zeroheight to surface live context during prototyping without switching apps. To start Monday: have an admin enable connectors for your organization, authenticate partner tools via OAuth, then reference them in Make prompts. This embeds real user research and data into design decisions, tightening collaboration between product, design, and engineering teams by grounding prototypes in actual customer context instead of assumptions.',
  'sources': [{'name': 'Figma', 'url': 'https://www.figma.com/release-notes/'},
   {'name': 'Figma Make: Manage connectors',
    'url': 'https://help.figma.com/hc/en-us/articles/36343926263703-Manage-connectors-in-Figma-Make'},
   {'name': 'Dovetail',
    'url': 'https://dovetail.com/changelog/dovetail-connector-figma-make'},
   {'name': 'Marvin',
    'url': 'https://help.heymarvin.com/en/articles/13773487-access-marvin-data-while-designing-in-figma'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1309_2026-02-21',
  'topic': 'Workflow improvement'},
 {'headline': 'Notion Agents Launch Next Week, Already Building Half Its Databases',
  'output': 'Notion will publicly launch custom AI agents next week that autonomously perform tasks across workspaces. Over 50% of Notion databases are now built by agents. The agents handle end-to-end workflows like generating status updates, building databases, and drafting documents across connected apps including Jira, Slack, and email. Product teams can automate cross-functional planning, research, and reporting that previously required manual page updates. Agents operate with your existing permissions and connect via expanded integrations. This matters now because it transforms knowledge work from manual coordination into agent-driven automation, relevant for teams planning workflows in Q1 and Q2 2026.',
  'sources': [{'name': 'Sources.news',
    'url': 'https://sources.news/p/notions-next-act'},
   {'name': 'Notion', 'url': 'https://www.notion.com/releases/2025-09-18'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1327_2026-02-21',
  'topic': 'Workflow improvement'},
 {'headline': 'Weaviate Cuts RAG Pipeline Coding With Agent Skills',
  'output': 'Weaviate has launched Agent Skills, an open-source repository that lets AI coding assistants like Claude, Cursor, and GitHub Copilot automate Weaviate tasks through a unified command surface. Software engineers building RAG pipelines can now guide agents to handle schema inspection, data imports, and retrieval workflows instead of writing custom scripts. You can start Monday by installing the skills via npx, spinning up a free 14-day Weaviate Cloud Sandbox, and running quickstart commands to integrate agents into your environment. The shift standardizes how teams prototype and debug, cutting manual integration work and accelerating demos across engineering, product, and marketing.',
  'sources': [{'name': 'Weaviate',
    'url': 'https://weaviate.io/blog/weaviate-agent-skills'},
   {'name': 'GlobeNewswire',
    'url': 'https://www.globenewswire.com/news-release/2026/02/21/3242244/0/en/Weaviate-Launches-Agent-Skills-to-Empower-AI-Coding-Agents.html'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1332_2026-02-21',
  'topic': 'Workflow improvement'},
 {'headline': 'WordPress Embeds AI Assistant for One-Prompt Content Edits',
  'output': "WordPress has launched an opt-in AI assistant built into its site editor and media library that edits text, translates content, adjusts fonts and layouts, and generates or edits images using Google's Nano Banana models. Marketers and designers can now update text, design, and visuals with a single natural-language prompt instead of manual edits across blocks and media assets. To start, enable AI tools in Site Settings, use the new Generate Image option in the Media Library, or type @ai in block notes for instant content suggestions. Sites built via WordPress' AI website builder have the assistant enabled by default, helping teams accelerate Q2 site management and campaign asset production.",
  'sources': [{'name': 'The Verge',
    'url': 'https://www.theverge.com/tech/880223/wordpress-launches-ai-assistant'},
   {'name': 'TechCrunch',
    'url': 'https://techcrunch.com/2026/02/17/wordpress-com-adds-an-ai-assistant-that-can-edit-adjust-styles-create-images-and-more/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1334_2026-02-21',
  'topic': 'Workflow improvement'},
 {'headline': 'Meta Bakes Manus AI Into Ads Manager Workflow',
  'output': 'Meta has embedded Manus AI directly into Ads Manager, eliminating the need for external tools or manual API work to generate reports and analyze audiences. Advertisers now access AI-powered campaign insights, automated reporting, and audience research through in-workflow prompts and a Tools menu shortcut. The integration is rolling out to all advertisers as of mid-February 2026. Marketing, data, and product teams can now collaborate faster inside a single platform. Starting Monday, enable the Manus shortcut, run a test automated report on a live campaign, and translate one AI-generated insight into an optimization task.',
  'sources': [{'name': 'Search Engine Land',
    'url': 'https://searchengineland.com/meta-adds-manus-ai-tools-into-ads-manager-469410'},
   {'name': 'Social Media Today',
    'url': 'https://www.socialmediatoday.com/news/meta-launches-manus-ai-integration-in-ads-manager/812288/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1338_2026-02-21',
  'topic': 'Workflow improvement'},
 {'headline': 'India mandates AI-generated content labels by February 20',
  'output': "India's new IT Rules 2026, effective February 20, mandate that all synthetic AI content (audio, video, images) must carry permanent labels and embedded metadata for traceability. Social media platforms must deploy user declaration flows, verification mechanisms, and remove unlawful AI content within three hours of a court or government notice, down from previous longer windows. Routine editing and educational materials are exempt. Platforms face enforcement penalties under existing Indian law. Implementation priority: build robust labeling infrastructure with embedded metadata and traceability chains to meet the three-hour takedown requirement and shortened seven-day grievance timelines.",
  'sources': [{'name': 'The Gazette of India',
    'url': 'https://egazette.gov.in/WriteReadData/2026/269993.pdf'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1302_2026-02-21',
  'topic': 'Report'},
 {'headline': '70% of Firms Use AI, Yet 80% See Zero Impact',
  'output': 'A new NBER study surveying nearly 6,000 executives across the US, UK, Germany, and Australia reveals a striking gap between AI adoption and results. Around 70% of firms actively use AI and two-thirds of executives spend 1.5 hours weekly on it, yet over 80% report no employment or productivity impact over the past three years. Firms forecast 1.4% productivity gains and 0.7% job cuts over the next three years, while employees expect net job growth. The takeaway: substantial ROI demands structured measurement frameworks, workflow redesign, and cross-functional governance, not just tool deployment.',
  'sources': [{'name': 'NBER', 'url': 'https://www.nber.org/papers/w34836'},
   {'name': 'Fortune',
    'url': 'https://fortune.com/2026/02/17/ai-productivity-paradox-ceo-study-robert-solow-information-technology-age/'}],
  'model_provider': 'claude',
  'news_date': '2026-02-21',
  'event_id': '1304_2026-02-21',
  'topic': 'Report'}]

for i in final_articles:
    if i['news_date'] >= '2026-02-21':
        save_article(i)
        print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [25]:
for i in final_summary_3:
    save_article(i)
    print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [110]:

fail = []
pas = 0

for i in final_summary_7:
    words = i['output'].split()
    word_count = len(words)
    if word_count > 100:
        print(f"Fail: {word_count}")
        fail.append(word_count)
    else:
        print(f"Pass: {word_count}")
        pas +=1

print(f"Fail: {len(fail)/len(final_summary_7)}")



Fail: 102
Fail: 107
Pass: 100
Pass: 100
Pass: 97
Fail: 101
Fail: 104
Pass: 97
Fail: 104
Pass: 97
Pass: 95
Pass: 100
Fail: 109
Pass: 100
Fail: 104
Pass: 99
Fail: 0.4375


In [223]:
import statistics as stats

In [111]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.

                TASK: Write a news article using ONLY the information provided by our research assistant. The research assistant will give 
                you outputs from 2 different types:
                1. Blogs/Articles/etc
                2. X/Twitter

                You need to ingest everything and take out the most relevant points to carefully write an article. By following this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                - Explain it in a simple way such that a non-tech person who has no background about this also understands its impact
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                ☑ There should be no em-dashes in the article
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:

                
                Research output from news/blogs: {i['deep_research_output']}

                Research output from X/Twitter: {i['x_output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_7.append(article_json)

        # save_article(article_json)

        print("✔️ Saved!")
        
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [118]:
test = [{
    "id": 224,
    "event_id": "1291_2026-02-20",
    "news_date": "2026-02-20",
    "x_output": "- Sarvam AI released two new large language models, Sarvam 30B and Sarvam 105B, on February 18, 2026.\n\n- Both models use Mixture of Experts (MoE) architecture and are built from scratch.\n\n- Sarvam 30B activates 1B non-embedded parameters per token.\n\n- Sarvam 30B was pretrained on 16 trillion tokens spanning code, web, multilingual, and mathematical data.\n\n- Sarvam 30B supports a 32K context window for long-running agentic interactions.\n\n- Sarvam 30B targets real-time applications like conversational AI and high-throughput workflows.\n\n- Sarvam 105B activates 9B parameters per token.\n\n- Sarvam 105B supports a 128K context window.\n\n- Sarvam 105B handles complex reasoning, agentic task completion, tool use, coding, mathematics, and science.\n\n- Sarvam AI builds full-stack generative AI for population-scale use.",
    "deep_research_output": "- Core development: Sarvam AI unveiled two indigenous LLMs, Sarvam-30B and Sarvam-105B, announced at the India AI Impact Summit 2026 in New Delhi, aimed at real-time use and advanced reasoning; MoE architecture with large context windows; open-sourcing planned. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Model architecture: Both models use a mixture-of-experts design that activates only a fraction of parameters at inference to improve efficiency. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Real-time capabilities: 30B targets production-ready, real-time conversational use with a 32,000-token context window; 105B targets more complex, multi-step reasoning with a 128,000-token window. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Training from scratch: 30B was pre-trained from scratch on about 16 trillion tokens; 105B trained on trillions of tokens across multiple Indian languages. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- IndiaAI Mission and compute: Training leveraged India's government-backed IndiaAI Mission compute, with infrastructure support from Yotta and technical support from Nvidia. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Open-source plan: Sarvam stated it intends to open-source the 30B and 105B models, though it did not specify whether training data or full training code would be disclosed. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Enterprise and ecosystem: The launch outlined an expanded enterprise stack, including Sarvam for Work and a conversational agent platform called Samvaad, plus a vision model for document understanding. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Indus consumer app: Two days after the model announcements, Sarvam released Indus, a consumer AI chat app powered by the 105B model, available in beta on iOS, Android, and the web; initial rollout limited to India with a waitlist. ([techcrunch.com](https://techcrunch.com/2026/02/20/indias-sarvam-launches-indus-ai-chat-app-as-competition-heats-up/?utm_source=openai))\n\n- Benchmark and language focus: The 105B model is positioned to perform on benchmarks relevant to Indian languages and enterprise tasks, with claims of competitive performance against open/closed models of similar size. ([business-standard.com](https://www.business-standard.com/technology/tech-news/sarvam-105b-model-sovereign-ai-india-foundation-model-launch-impact-summit-126021900551_1.html?utm_source=openai))\n\n- Industry coverage: Coverage of the announcement and related products appeared in TechCrunch (Feb 18, 2026), Economic Times (ETtech, Feb 18, 2026), Fortune India (Feb 18, 2026), and The Times of India (Feb 19, 2026), among others. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))"
  }]

In [119]:
claude_article_builder(test)

{'headline': 'Indian Startup Sarvam AI Drops Two Homegrown LLMs', 'output': "Sarvam AI just unveiled two indigenous large language models, Sarvam-30B and Sarvam-105B, at India's AI Impact Summit 2026. Both were trained from scratch on trillions of tokens across multiple Indian languages using government-backed compute infrastructure. The 30B model targets real-time conversations with a 32,000-token context window, while the 105B handles complex reasoning with a massive 128,000-token window. Both use mixture-of-experts architecture, activating only needed parameters for efficiency. Sarvam plans to open-source both models and launched Indus, a consumer chat app powered by the 105B. This positions India to compete in the global AI race with sovereign, multilingual models tailored for local needs.", 'sources': [{'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/'}, {'name': 'TechCrunch', 'url'

In [ ]:
[
  {
    "headline": "India's Sarvam AI Drops Two Homegrown LLMs, Goes Open-Source",
    "output": "Sarvam AI just unveiled two made-in-India language models at the India AI Impact Summit: 
    Sarvam-30B for real-time chat and Sarvam-105B for complex reasoning. Both use mixture-of-experts architecture, 
    activating only some parameters to stay efficient while handling massive context windows (32K and 128K tokens). 
    Trained from scratch on trillions of tokens across Indian languages using government-backed compute, they 
    rival global models but understand local languages better. Sarvam plans to open-source both and launched Indus, 
    a consumer chat app powered by 105B. This could make advanced AI accessible across India's linguistic diversity, 
    challenging Western-dominated models with sovereign, multilingual alternatives."
  },
  {
    "headline": "Stanford and AWS Team Up for AI Marketing Lab",
    "output": "Stanford HAI and AWS just launched the Stanford and AWS Marketing Science Lab to revolutionize how companies measure marketing effectiveness using AI. The collaboration brings together Stanford's Institute for Human-Centered AI, Data Science department, Graduate School of Business, and AWS cloud infrastructure. Led by top economists including Nobel laureate Guido Imbens and Susan Athey, the lab will tackle causal inference, AI measurement tools, and B2B customer journey analysis. Think smarter ad spend decisions powered by algorithms instead of guesswork. The initiative plans to release published research, open-source code, and prototypes, potentially reshaping how businesses justify their marketing budgets and adopt AI tools at scale."
  },
  {
    "headline": "ChatGPT's New Lockdown Mode Blocks Live Web Browsing",
    "output": "OpenAI just dropped Lockdown Mode for ChatGPT, a high-security setting rolling out to Enterprise, Edu, Healthcare, and Teachers accounts, with consumer access coming soon. Designed for executives and security teams worried about data leaks, it tackles prompt-injection attacks by blocking live web requests and restricting external app access. Instead of browsing the open internet, ChatGPT can only pull cached content within OpenAI's network. Admins can customize which tools stay active per user. Paired with new Elevated Risk labels flagging dangerous features, this upgrade could reshape how organizations trust AI with sensitive workflows and confidential information."
  },
  {
    "headline": "Google's Lyria 3 Turns Text Into Music in Gemini",
    "output": "Google DeepMind just dropped Lyria 3 in beta, letting Gemini users create 30-second tracks from text, photos, or videos with auto-generated lyrics and cover art. Think of it as Instagram for your ears: describe a vibe or upload a sunset pic, and get a custom soundtrack perfect for TikTok or Reels. The AI handles genre, mood, tempo, and vocals while blocking artist copycats. Each track gets watermarked via SynthID so you can spot AI audio. Available now on desktop (mobile coming soon) for users 18+ in eight languages, with YouTube Dream Track integration expanding creator access. As AI music tools proliferate, expect short-form content to become hyper-personalized sonic experiences."
  },
  {
    "headline": "Insurance Startup Qumis Raises $4.3M for Lawyer-Level AI",
    "output": "Chicago-based Qumis just closed an oversubscribed $4.3 million seed round led by MTech Capital, bringing total funding to $6.75 million. The startup uses attorney-trained AI to read complex commercial insurance policies and deliver citation-backed insights with transparent reasoning, replacing tedious manual reviews. Major brokers like NFP, an Aon company, have already scaled from pilot teams to hundreds of users. While most insurance AI tackles workflow automation, Qumis is betting on specialized coverage intelligence. The funds will expand its go-to-market team and deepen product capabilities, positioning it as AI moves beyond document processing into nuanced legal interpretation for underwriting and claims."
  },
  {
    "headline": "OpenAI and Tata Group Join Forces for India's AI Revolution",
    "output": "OpenAI and Tata Group have announced a massive strategic partnership to accelerate AI transformation across India and beyond. The deal includes deploying ChatGPT Enterprise to hundreds of thousands of TCS employees, building sovereign AI data centers starting at 100 MW (scalable to 1 GW), and developing industry-specific agentic AI solutions. TCS becomes OpenAI's first HyperVault customer, ensuring data stays local and secure. The partnership also brings 100,000 ChatGPT Edu licenses and AI training for one million Indian youth. With new OpenAI offices opening in Mumbai and Bengaluru by 2026, this positions India as a global AI powerhouse while giving enterprises tools to compete in the AI-first economy."
  },
  {
    "headline": "Fractal's Vaidya 2.0 Beats GPT-5 on Healthcare Benchmark",
    "output": "Fractal just launched Vaidya 2.0, a healthcare AI model that's crushing the competition. It scored 50.1 on OpenAI's notoriously tough HealthBench test, becoming the first model to cross 50 and outperforming both GPT-5 and Google's Gemini Pro 3. Think of it as an AI doctor's assistant that can triage emergencies, check symptoms, and guide patients through treatment. Unlike typical AI models that just recall medical facts, Vaidya 2.0 actually reasons through complex healthcare scenarios. Part of India's $1.2 billion AI push, it's designed specifically for healthcare systems in developing regions, bridging the gap between raw patient data and actual medical action."
  },
  {
    "headline": "OpenAI Building Smart Speakers, Glasses After $6.5B Acquisition",
    "output": "OpenAI is jumping into consumer hardware with a team of over 200 people developing AI-powered devices, including smart speakers, glasses, and even a smart lamp. This follows their massive $6.5 billion acquisition of io Products, Jony Ive's design firm. The first product, a camera-equipped smart speaker priced at $200 to $300, won't ship until February 2027 at the earliest, while smart glasses are slated for 2028. This pivot from pure software to physical products puts OpenAI in direct competition with Meta's successful Ray-Ban glasses and upcoming wearables from Apple and Google, as AI companies race to embed intelligence into everyday objects around your home."
  },
  {
    "headline": "OpenAI and Microsoft Join UK's £27M AI Safety Push",
    "output": "OpenAI and Microsoft have officially joined the UK's AI Security Institute's international coalition to keep AI development safe and under human control. The Alignment Project, a UK-led initiative, now boasts over £27 million in funding supporting roughly 60 research projects across eight countries. OpenAI is chipping in £5.6 million, joining partners like Anthropic, AWS, and Canada's CIFAR. Think of it as a global insurance policy: researchers get grants, computing power, and mentorship to ensure future AI systems don't go rogue. With a second funding round opening this summer, the coalition aims to build public trust and enable safer AI deployment in critical services worldwide."
  },
  {
    "headline": "General Catalyst Drops $5 Billion Bet on India",
    "output": "General Catalyst just pledged $5 billion to India over five years, a massive fivefold jump from its previous $500 million to $1 billion commitment. Announced at the India AI Impact Summit, the funds will flow into AI, healthcare, defense tech, fintech, and consumer startups. GC isn't chasing cutting-edge AI model development. Instead, it's betting on deploying AI solutions at scale in the real world, where India's vast market shines. With CEO Neeraj Arora steering the charge and a portfolio already featuring Zepto and PB Health, this signals India's emergence as a global AI deployment powerhouse, not just a talent hub."
  },
  {
    "headline": "Google Drops Gemini 3.1 Pro With Wild Reasoning Upgrade",
    "output": "Google just launched Gemini 3.1 Pro, a significantly smarter AI model built for complex problems that need more than quick answers. It scored 77.1% on the ARC-AGI-2 benchmark, literally doubling the reasoning power of its predecessor. The upgrade rolls out today across the Gemini app, NotebookLM, API, and enterprise tools. What makes it wild? It can generate animated website code from text, build live aerospace dashboards tracking the ISS, and even code 3D bird simulations with hand-tracking. This positions Google to compete harder in agentic AI workflows where models actually solve multi-step problems autonomously, not just chat."
  },
  {
    "headline": "PhonePe Lets You Pay Friends Just By Talking",
    "output": "PhonePe just launched AI-powered voice and text search built on Microsoft Foundry, letting its 650 million users skip menus entirely. Say \"Pay Hemanth 20 rupees\" and it auto-selects the contact, or ask \"Gold price\" to jump straight there. The feature uses intent-based routing instead of traditional navigation, understanding what you want and taking you there instantly. It runs on a hybrid on-device and cloud model, keeping your transaction data locked within PhonePe's environment. Rolling out in phases across India via the search bar, help center, and history tab, it signals a shift toward conversational interfaces in fintech."
  },
  {
    "headline": "India Gets 8-Exaflop AI Supercomputer From UAE Partnership",
    "output": "Abu Dhabi's G42 and MBZUAI, teaming up with Cerebras and India's C-DAC, just announced an 8-exaflop AI supercomputer for India at the India AI Impact Summit 2026. That's mind-blowing computing power for training massive AI models tailored to Indian needs. The system will be hosted entirely in India with full data sovereignty, meaning all your data stays within national borders under Indian governance. Once live, universities, startups, SMEs, and government ministries get access to this beast. This positions India as a serious player in sovereign AI infrastructure, reducing dependence on foreign cloud giants while accelerating homegrown AI innovation at scale."
  },
  {
    "headline": "Meta's Smartwatch Is Back: 2026 AI Wearable Incoming",
    "output": "Meta is reviving its smartwatch ambitions with Malibu 2, slated for late 2026. The device will pack health tracking and a built-in Meta AI assistant, taking on the Apple Watch directly. This marks a comeback after Meta killed a similar project in 2022 during Reality Labs budget cuts. The smartwatch fits into Meta's broader wearables strategy, potentially syncing with its Ray-Ban smart glasses, which shipped nearly 6 million units in 2025. Meta hasn't officially commented. If successful, the AI-powered watch could anchor an ecosystem where your glasses, wrist, and phone work together seamlessly, pushing wearables beyond fitness into ambient computing."
  },
  {
    "headline": "Nvidia Ditches $100B Chip Deal for $30B OpenAI Stake",
    "output": "Nvidia is swapping its massive $100 billion multi-year chip supply agreement for a cleaner $30 billion direct equity investment in OpenAI. This shift makes Nvidia an actual owner, not just a supplier, in the AI giant currently valued at $830 billion. OpenAI's hunting for over $100 billion total this round, with SoftBank and Amazon joining the party. Plot twist: OpenAI plans to spend much of that fresh cash buying Nvidia chips anyway. This move signals Nvidia wants skin in the game beyond hardware sales, positioning itself as a stakeholder in AI's future rather than just the picks-and-shovels provider during the gold rush."
  }
]